<a href="https://colab.research.google.com/github/Bambillo1/elecomsCoding_academy/blob/main/Copy_of_Enhancing_Electricity_Theft_Detection_and_Mitigation_in_Smart_Grid_Through_Hybrid_CNN_RF_and_Advanced_Baseline_Neural_Network_Techniques_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Libraries import
import tensorflow as tf
from tensorflow import keras
import os
import tempfile

import matplotlib as mp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler


import tensorflow as tf
import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from imblearn.over_sampling import SMOTE, ADASYN
from tensorflow.keras import Model

from keras.callbacks import ModelCheckpoint,EarlyStopping,CSVLogger, LearningRateScheduler,ReduceLROnPlateau
from tensorflow.keras.metrics import binary_crossentropy, TruePositives, TrueNegatives, FalsePositives, FalseNegatives, BinaryAccuracy, Precision, Recall, AUC

from keras.callbacks import ModelCheckpoint
#from plot_keras_history import show_history, plot_history
#from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import plot_model
#from keras.utils.vis_utils import plot_model
#!pip install scikeras
#!pip install plot-keras-history
#from scikeras.wrappers import KerasClassifier

In [ ]:
#Setting parameters for plots
mp.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

#Mounting data on gdrive


In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')

Link data residing in gdrive to colab using this link from the data(in gdrive)

In [ ]:
!gdown  --id 1OIz8U3aymDeGDUIl83vyTFCPyfLulSgM

Using read_csv to read in data into a dataframe

In [ ]:
df = pd.read_csv('data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

#Proportion of flag (1) and not flag(0)

In [ ]:
#Proportion of flag (1) and not flag(0)

one = df[df['FLAG'] == 1].shape[0]
zero = df[df['FLAG'] == 0].shape[0]

total = zero + one

In [ ]:
total

In [ ]:
print(one / (one + zero) * 100, '% of customers flagged.')

print(zero / (one + zero) * 100, '% of customers unflagged.')

Visualize data using pie chart

In [ ]:
df["FLAG"].value_counts().plot(kind = 'pie',explode=[0, 0.1],figsize=(6, 6),autopct='%1.1f%%',shadow=True)
plt.title("Fraudulent and Non-Fraudulent Distribution",fontsize=20)
plt.legend(["zero", "one"])
plt.show()

Check first 10 rows of the df

In [ ]:

df.head(10)

Append other columns except for 'CONS_NO','FLAG' columns into a list named Ib

In [ ]:
l=df.columns
la=['CONS_NO','FLAG']
lb=[]
for i in l:
    if i not in la:
        lb.append(i)

Format the date in year/month/day mode for all columns and store them in a list fdatesdates

In [ ]:

import datetime
dates = [datetime.datetime.strptime(ts, "%Y/%m/%d") for ts in lb]
#dates.sort()
fdatesdates = [datetime.datetime.strftime(ts, "%Y/%m/%d") for ts in dates]

Insert 0 in all rows of column FLAG, same with CON_NO

In [ ]:
fdatesdates.insert(0,"FLAG")
fdatesdates.insert(0,"CONS_NO")

df.columns=fdatesdates

Using sort method to sort dates in ascending order

In [ ]:
import datetime
dates = [datetime.datetime.strptime(ts, "%Y/%m/%d") for ts in lb]
dates.sort()
sorteddates = [datetime.datetime.strftime(ts, "%Y/%m/%d") for ts in dates]


Concatenate sorted dates AND columns CONS_NO and FLAG

In [ ]:
cols=df.columns.tolist()[0:2]+sorteddates
df=df[cols]

Fill all columns with there respective observations

In [ ]:
import math
train_df=df
l=train_df["2014/01/01"]
l1=train_df["2014/01/01"]
l=np.asarray(l).tolist()
l1=np.asarray(l1).tolist()

l2=[]
for i in range(len(l)):
    if math.isnan(l[i]):
        if math.isnan(l1[i]):
            l2.append(0)
        else:
            l2.append(l1[i]/2)
    else:
        l2.append(l[i])
train_df["2014/01/01"]=l2

train_df.head()
l=train_df["2016/10/31"]
l1=train_df["2016/10/31"]
l=np.asarray(l).tolist()
l1=np.asarray(l1).tolist()

l2=[]
for i in range(len(l)):
    if math.isnan(l[i]):
        if math.isnan(l1[i]):
            l2.append(0)
        else:
            l2.append(l1[i]/2)
    else:
        l2.append(l[i])
train_df["2016/10/31"]=l2


l=train_df.columns
la=['CONS_NO','FLAG']
lbx=[]
for i in l:
    if i not in la:
        lbx.append(i)





CNS_NO is a categorical columns that contains string objects representing each customer. It is of no use in the dataframe for modelling, thus, dropped or removed.

In [ ]:
df = df.drop('CONS_NO', axis = 1)

Using interpolate method for replacing missing values

In [ ]:
df=df.interpolate(method ='linear', limit_direction ='forward')
df=df.interpolate(method ='linear', limit_direction ='backward')

Splitting data into train, test and validation sets with test size of 20%, validation 20% and train of 60%

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, stratify = df['FLAG'])
df_train, df_val = train_test_split(df, test_size=0.2, stratify = df['FLAG'])

In [ ]:
#Check first 5 rows of df
df_train.head()

In [ ]:
df_test

In [ ]:
df_val

In [ ]:
# Convert dataframe into  numpy arrays of labels and features.
df_train_labels = np.array(df_train.pop('FLAG'))   #pop function means remove FLAG from the train data

In [ ]:
df_val_labels = np.array(df_val.pop('FLAG'))  #pop function means remove FLAG from the val data

In [ ]:
df_test_labels = np.array(df_test.pop('FLAG'))#pop function means remove FLAG from the test data

In [ ]:
bin_train_labels = df_train_labels != 0 #Where df_train_labels is not equal to zero

In [ ]:
df_train_features = np.array(df_train) #This is now  a numpy array of train data

In [ ]:
df_val_features = np.array(df_val)   ##This is now  a numpy array of val data

In [ ]:
df_test_features = np.array(df_test)#This is now  a numpy array of test data

In [ ]:
df_train_features.shape #Check no of rows and columns of train data

Use MinMaxScaler to normalize all features i -: Uniform Scale

In [ ]:
scale_features = MinMaxScaler()

In [ ]:
df_train_features = scale_features.fit_transform(df_train_features)

In [ ]:
df_val_features = scale_features.transform(df_val_features)

In [ ]:
df_test_features = scale_features.transform(df_test_features)

In [ ]:
df_train_features = np.clip(df_train_features, -5, 5) #Remove any observation less and equal to -5 and less than and greater than 5

In [ ]:
df_val_features = np.clip(df_val_features, -5, 5)

In [ ]:
df_test_features = np.clip(df_test_features, -5, 5)

In [ ]:
print(' Shape of training labels: ', df_train_labels.shape)
print(' Shape of validation labels: ', df_val_labels.shape)
print(' Shape of test labels: ', df_test_labels.shape)

Print the shapes of all features

In [ ]:
print('Shape of training features : ', df_train_features.shape)
print('Shape of validation features: ', df_val_features.shape)
print('Shape of test features: ', df_test_features.shape)

Build a function for performance metrics and baseline model

In [ ]:
METRICS = [
      TruePositives(name='tp'),
      FalsePositives(name='fp'),
      TrueNegatives(name='tn'),
      FalseNegatives(name='fn'),
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
      AUC(name='prc', curve='PR'),
                        ]

def make_model(metrics=METRICS, output_bias=None):

  if output_bias is not None:

    output_bias = tf.keras.initializers.Constant(output_bias)

  model = keras.Sequential([keras.layers.Dense(32, activation='relu',input_shape=(df_train_features.shape[-1],)),

      keras.layers.Flatten(),

      keras.layers.Dense(32, activation = 'relu', kernel_initializer = 'he_uniform'),

      keras.layers.Dropout(0.7),

      keras.layers.Dense(64, activation = 'relu', kernel_initializer = 'he_uniform'),

      keras.layers.Dropout(0.7),

      keras.layers.Dense(1, activation='sigmoid',bias_initializer=output_bias),
  ])

  model.compile(optimizer=Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=metrics)

  return model

Notice that the model is fit using a larger than default batch size of 2048, this is important to ensure that each batch has a decent chance of containing a few positive samples. If the batch size was too small, they would likely have no fraudulent transactions to learn from.

#Baseline Model
early_stopping is when there is convergence during training-It means training stops where same scores are being repeated at every epoch

In [ ]:
epochs = 100  # no of iterations the neurons tranverses the network
batch_size = 2048 # Each batch of data that are entering the network at a time

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_prc', verbose=1, patience=10, mode='max', restore_best_weights=True)

In [ ]:
model = make_model() # invoking make_model function created above
model.summary()

We are now predicting the probabilites using the intial weights assigned and the initial predictions are not that generalized. Many of the values are predicted as fraud transactions (probability > 0.5). We can see that the class imbalance is not learnt by the model and it will take a few epochs for the model to learn that there are very less fraud transactions.

In [ ]:
model.predict(df_train_features[:15])

#Understanding Initial Loss
Evaluate the model of training data and check the initial loss during training.
When initializing the biases of the neural network's final layer (the layer connected to the sigmoid activation) to their default values,
it is often set such that the initial predicted probability is around 0.5 for both classes. This makes sense because, in the beginning,
the model has no knowledge about the data distribution, and it's a reasonable assumption that both classes are equally likely.
According to this if for one sample we assume the probability as 0.5 and calculate the loss using binary crossentropy, out initial loss should be:
log(2) = 0.69314

In [ ]:
returns = model.evaluate(df_train_features, df_train_labels, batch_size=batch_size, verbose=0)
print("The initial model loss is {:0.5f}".format(returns[0]))

Bias of the model is the logarithm of proportion of positive divided by the negative  class

In [ ]:
first_bias = np.log([one/zero])
first_bias

Using the first bias above in the baseline model function to make prediction on train data

In [ ]:
model = make_model(output_bias=first_bias)
model.predict(df_train_features[:15])

This shows that our loss remains stagnant as there is no change in values of initial loss and returns_later below

In [ ]:
returns_later = model.evaluate(df_train_features, df_train_labels, batch_size=batch_size, verbose=0)
print("Loss: {:0.5f}".format(returns_later[0]))

#Checkpoint the initial weights¶
To make the various training runs more comparable, keep this initial model's weights in a checkpoint file, and load them into each model before training: This statement means the model should be trained and save somewhere. If we need to start it again, we check where it's saved and import it into the system for more training.

Before moving on, confirm quick that the careful bias initialization actually helped.

Train the model for 20 epochs, with and without this careful initialization, and compare the losses:

In [ ]:
first_weights = os.path.join(tempfile.mkdtemp(), 'first_weights')
model.save_weights(first_weights)

Assign 0 to be weight of the last layer(-1) and train the data on train data and validate with validation data

In [ ]:
model = make_model()
model.load_weights(first_weights)
model.layers[-1].bias.assign([0.0])
zero_bias_history = model.fit(df_train_features, df_train_labels, batch_size=batch_size, epochs=20, validation_data=(df_val_features, df_val_labels),  verbose=0)

Train the data as above but by assigning1 as weight of the last layer

In [ ]:
model = make_model()
model.load_weights(first_weights)
careful_bias_history = model.fit(df_train_features, df_train_labels, batch_size=batch_size, epochs=20,  validation_data=(df_val_features, df_val_labels),  verbose=0)

Create a plot function to plot zero_bias and creful_bias trainings

In [ ]:
def plot_loss_model(history, label, n):
  # Use a log scale on y-axis to show the wide range of values.
  plt.semilogy(history.epoch, history.history['loss'], color=colors[n], label='Train ' + label)
  plt.semilogy(history.epoch, history.history['val_loss'], color=colors[n], label='Val ' + label, linestyle="--")
  plt.xlabel('Epoch')
  plt.ylabel('Loss')

Invoking the function to plot the two biases

In [ ]:
plot_loss_model(zero_bias_history, "Zero Bias", 0)
plot_loss_model(careful_bias_history, "Careful Bias", 1)

From the checkpoint, baseline model we saved earlier is now eing loaded for training

In [ ]:
model = make_model()
model.load_weights(first_weights)
baseline_model_history = model.fit(df_train_features, df_train_labels, batch_size=batch_size, epochs=epochs, callbacks=[early_stopping], validation_data=(df_val_features, df_val_labels))

This function plots model metric function defined earlier: precision,recall, prc, loss

In [ ]:
def plot_model_metrics(history):
  metrics = ['loss', 'prc', 'precision', 'recall']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric], color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.9,1])
    else:
      plt.ylim([0,1])

    plt.legend()

In [ ]:
plot_model_metrics(baseline_model_history)

Make pediction with baselime model on both train and test data

In [ ]:
train_predictions_baseline = model.predict(df_train_features, batch_size=batch_size)
test_predictions_baseline = model.predict(df_test_features, batch_size=batch_size)

This function plots confusion matrix forthe baseline model

In [ ]:
def plot_cm_model(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(8,8))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Non-Theft correctly Detected (TN): ', cm[0][0])
  print('Non-theft Incorrectly Detected (FP): ', cm[0][1])
  print('Theft incorrectly Detected (FN): ', cm[1][0])
  print('Theft correctly Detected (TP): ', cm[1][1])
  print('Total theft Detections: ', np.sum(cm[1]))

#Evaluate your model on the test dataset and display the results for the metrics you created above:
If the model had predicted everything perfectly, this would be a diagonal matrix where values off the main diagonal, indicating incorrect predictions, would be zero. In this case the matrix shows that you have relatively few false positives, meaning that there were relatively few legitimate connections that were incorrectly flagged. However, you would likely want to have even fewer false negatives despite the cost of increasing the number of false positives. This trade off may be preferable because false negatives would allow fraudulent connections to go through, whereas false positives may cause an email to be sent to a customer to ask them to verify their energy card activity.

In [ ]:
baseline_model_results = model.evaluate(df_test_features, df_test_labels, batch_size=batch_size, verbose=0)
for name, value in zip(model.metrics_names, baseline_model_results):
  print(name, ': ', value)
print()

plot_cm_model(df_test_labels, test_predictions_baseline)

This function plots roc curve of the baseline model

In [ ]:
def plot_roc_model(name, labels, predictions, **kwargs):
  fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

  plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
  plt.xlabel('False positives [%]')
  plt.ylabel('True positives [%]')
  plt.xlim([-0.5,100])
  plt.ylim([40,100.5])
  plt.grid(True)
  ax = plt.gca()
  ax.set_aspect('equal')

This plot is useful because it shows, at a glance, the range of performance the model can reach just by tuning the output threshold.

Invoking roc_model function with real arguments

In [ ]:
plot_roc_model("Train Baseline_Model", df_train_labels, train_predictions_baseline, color=colors[0])
plot_roc_model("Test Baseline_Model", df_test_labels, test_predictions_baseline, color=colors[0], linestyle='--')
plt.legend(loc='upper right');

This function plots prc of the baseline model

In [ ]:
def plot_model_prc(name, labels, predictions, **kwargs):
    precision, recall, _ = sklearn.metrics.precision_recall_curve(labels, predictions)

    plt.plot(precision, recall, label=name, linewidth=2, **kwargs)
    plt.xlabel('Precision')
    plt.ylabel('Recall')
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

Classifiers often face challenges when trying to maximize both precision and recall, which is especially true when working with imbalanced datasets. It is important to consider the costs of different types of errors in the context of the problem you care about. In this example, a false negative (a fraudulent transaction is missed) may have a financial cost, while a false positive (a transaction is incorrectly flagged as fraudulent) may decrease user happiness.

Invoking model_prc function with real arguments

In [ ]:
plot_model_prc("Train Baseline", df_train_labels, train_predictions_baseline, color=colors[0])
plot_model_prc("Test Baseline", df_test_labels, test_predictions_baseline, color=colors[0], linestyle='--')
plt.legend(loc='lower right');

##Calculate class weights
The goal is to identify fraudulent connections, but you don't have very many of those positive samples to work with, so you would want to have the classifier heavily weight the few examples that are available. You can do this by passing Keras weights for each class through a parameter. These will cause the model to "pay more attention" to examples from an under-represented class (1).

In [ ]:
# Scaling by total/2 helps keep the loss to a similar value.
# The sum of the weights of all examples stays the same.
weight_0 = (1 / zero) * (total / 2.0)
weight_1 = (1 / one) * (total / 2.0)

class_weight = {0: weight_0, 1: weight_1}

print('Weight for class negative (0): {:.2f}'.format(weight_0))
print('Weight for class positive (1): {:.2f}'.format(weight_1))

Now try re-training and evaluating the model with class weights to see how that affects the predictions.

Using class_weights changes the range of the loss. This may affect the stability of the training depending on the optimizer. Optimizers whose step size is dependent on the magnitude of the gradient, like tf.keras.optimizers.SGD, may fail. The optimizer used here, tf.keras.optimizers.Adam, is unaffected by the scaling change. Also note that because of the weighting, the total losses are not comparable between the two models.

In [ ]:
weighted_model = make_model()
weighted_model.load_weights(first_weights)

weighted_history = weighted_model.fit(df_train_features,df_train_labels,batch_size=batch_size, epochs=epochs, callbacks=[early_stopping], validation_data=(df_val_features, df_val_labels), class_weight=class_weight)


In [ ]:
plot_model_metrics(weighted_history)

Make prediction with class weight calculate above on both train and test data

In [ ]:
train_predictions_weighted = weighted_model.predict(df_train_features, batch_size=batch_size)
test_predictions_weighted = weighted_model.predict(df_test_features, batch_size=batch_size)

Print results using confusion matrix for class weight

In [ ]:
weighted_results = weighted_model.evaluate(df_test_features, df_test_labels, batch_size=batch_size, verbose=0)
for name, value in zip(weighted_model.metrics_names, weighted_results):
  print(name, ': ', value)
print()

plot_cm_model(df_test_labels, test_predictions_weighted)

Also plot roc for the baseline model and weighted mdel using train and test data

In [ ]:
plot_roc_model("Train Baseline_Model", df_train_labels, train_predictions_baseline, color=colors[0])
plot_roc_model("Test Baseline_Model", df_test_labels, test_predictions_baseline, color=colors[0], linestyle='--')
plot_roc_model("Train Weighted_Model", df_train_labels, train_predictions_weighted, color=colors[1])
plot_roc_model("Test Weighted_Model", df_test_labels, test_predictions_weighted, color=colors[1], linestyle='--')
plt.legend(loc='upper right');

Invoking prc function for baseline and weighted models using train and test data

In [ ]:
plot_model_prc("Train Baseline_Model", df_train_labels, train_predictions_baseline, color=colors[0])
plot_model_prc("Test Baseline_Model", df_test_labels, test_predictions_baseline, color=colors[0], linestyle='--')

plot_model_prc("Train Weighted_Model", df_train_labels, train_predictions_weighted, color=colors[1])
plot_model_prc("Test Weighted_Model", df_test_labels, test_predictions_weighted, color=colors[1], linestyle='--')


plt.legend(loc='lower right');

#Oversample the minority class

In [ ]:
one_features = df_train_features[bin_train_labels]      #train data for only where FLAG == 1
zero_features = df_train_features[~bin_train_labels]   #train data for only where FLAG == 0

In [ ]:
one_labels = df_train_labels[bin_train_labels]#train LABELS for only where FLAG == 1
zero_labels = df_train_labels[~bin_train_labels]#train LABELS for only where FLAG == 0

In [ ]:
(one_labels[:20], zero_labels[:20]) #Print first 20 for 1 and 0 labels

You can balance the dataset manually by choosing the right number of random indices from the positive examples:¶

Using numpy random module to shuffle the data and make some choice of 1 and 0 features.

This step tries to oversample the minority 1 class to e equal to the majority 0 class.

In [ ]:
ids = np.arange(len(one_features))
choices = np.random.choice(ids, len(zero_features))

In [ ]:
res_one_features = one_features[choices]
res_one_labels = one_labels[choices]

res_one_features.shape

Adding or concatenating 1 and 0 features together gives us a resampled features of 62010 rows and 1034 columns

With tf.data from tensorflow, the easiest way to produce balanced examples is to start with a one and a zero dataset, and concatenate them.

In [ ]:
resampled_features = np.concatenate([res_one_features, zero_features], axis=0)
resampled_labels = np.concatenate([res_one_labels, zero_labels], axis=0)

order = np.arange(len(resampled_labels))
np.random.shuffle(order)
resampled_features = resampled_features[order]
resampled_labels = resampled_labels[order]


In [ ]:
resampled_features.shape

Using tensorflow, create a function that shuffles our oversampled data and returns a shuffled dataset.

In [ ]:
BUFFER_SIZE = 100000

def make_ds(features, labels):
  ds = tf.data.Dataset.from_tensor_slices((features, labels))
  ds = ds.shuffle(BUFFER_SIZE).repeat()
  return ds

one_ds = make_ds(one_features, one_labels)
zero_ds = make_ds(zero_features, zero_labels)

Print a sample of a row from the dataset for feature and label

In [ ]:
for features, label in one_ds.take(1):
  print("Features:\n", features.numpy())
  print()
  print("Label: ", label.numpy())

Merge the two together using tf.data.Dataset.sample_from_datasets:

In [ ]:
resampled_ds = tf.data.Dataset.sample_from_datasets([one_ds, zero_ds], weights=[0.5, 0.5])
resampled_ds = resampled_ds.batch(batch_size).prefetch(2)

In [ ]:
for features, label in resampled_ds.take(1):
  print(label.numpy().mean())

To use this dataset, you'll need the number of steps per epoch.

epoch here is the number of batches required to see each negative example once:

In [ ]:
resampled_steps_per_epoch = np.ceil(2.0*zero/batch_size)
resampled_steps_per_epoch

## Train on the oversampled data
Now try training the model with the resampled data set instead of using class weights to see how these methods compare.

Note: Because the data was balanced by replicating the positive examples, the total dataset size is larger, and each epoch runs for more training steps.
Invoke baseline model function with oversampled data

In [ ]:
resampled_model = make_model()
resampled_model.load_weights(first_weights)

In [ ]:
# Reset the bias to zero, since this dataset is balanced.
output_layer = resampled_model.layers[-1]
output_layer.bias.assign([0])

In [ ]:
val_ds = tf.data.Dataset.from_tensor_slices((df_val_features, df_val_labels)).cache()
val_ds = val_ds.batch(batch_size).prefetch(2)

In [ ]:
resampled_history = resampled_model.fit(resampled_ds, epochs=epochs, steps_per_epoch=resampled_steps_per_epoch, callbacks=[early_stopping], validation_data=val_ds)

When training the model batch-wise, as you did here, the oversampled data provides a smoother gradient signal: Instead of each positive example being shown in one batch with a large weight, they're shown in many different batches each time with a small weight.

This smoother gradient signal makes it easier to train the model.

In [ ]:
plot_model_metrics(resampled_history)

Make prediction with train and test data on oversampled model

In [ ]:
train_predictions_resampled = resampled_model.predict(df_train_features, batch_size=batch_size)
test_predictions_resampled = resampled_model.predict(df_test_features, batch_size=batch_size)

Evaluate the oversampled model with test data

In [ ]:
resampled_results = resampled_model.evaluate(df_test_features, df_test_labels,  batch_size=batch_size, verbose=0)

Calling plot_cm_model with test data and predictions from oversampled model

In [ ]:
for name, value in zip(model.metrics_names, resampled_results):
  print(name, ': ', value)
print()

plot_cm_model(df_test_labels, test_predictions_resampled)

Because training is easier on the balanced data, the above training procedure may overfit quickly.

So break up the epochs to give the tf.keras.callbacks.EarlyStopping finer control over when to stop training.

In [ ]:
resampled_model = make_model()
resampled_model.load_weights(first_weights)

Increase number of epochs by multiplying by 10 = 100*10 and train the oversampled model again

In [ ]:
# Reset the bias to zero, since this dataset is balanced.
output_layer = resampled_model.layers[-1]
output_layer.bias.assign([0])
resampled_history = resampled_model.fit(resampled_ds, steps_per_epoch=20, epochs=10*epochs, callbacks=[early_stopping], validation_data=(val_ds))

Re-plot the metrics for the re-trained oversampled model above

In [ ]:
plot_model_metrics(resampled_history)

Make prediction for the oversampled model using train and test data

In [ ]:
train_predictions_resampled = resampled_model.predict(df_train_features, batch_size=batch_size)
test_predictions_resampled = resampled_model.predict(df_test_features, batch_size=batch_size)

Evaluate the re-oversampled model with test data

In [ ]:
resampled_results = resampled_model.evaluate(df_test_features, df_test_labels, batch_size=batch_size, verbose=0)

Print the results of all metrics for the re-oversampled model

In [ ]:
for name, value in zip(resampled_model.metrics_names, resampled_results):
  print(name, ': ', value)
print()

plot_cm_model(df_test_labels, test_predictions_resampled)

Invoke plot_roc_model function on predictions with train data of baseline and test data of  baseline model





In [ ]:
plot_roc_model("Train Baseline_Model", df_train_labels, train_predictions_baseline, color=colors[0])
plot_roc_model("Test Baseline_Model", df_test_labels, test_predictions_baseline, color=colors[0], linestyle='--')
plt.legend(loc='lower right');

Invoke plot_roc_model function on train data of the class weight and of the test data

In [ ]:
plot_roc_model("Train Weighted_Model", df_train_labels, train_predictions_weighted, color=colors[1])
plot_roc_model("Test Weighted_Model", df_test_labels, test_predictions_weighted, color=colors[1], linestyle='--')
plt.legend(loc='lower right');

Invoke plot_roc_model function on train data of the resampled and of the test data of the resampled model

In [ ]:
plot_roc_model("Train Resampled_Model", df_train_labels, train_predictions_resampled, color=colors[2])
plot_roc_model("Test Resampled_Model", df_test_labels, test_predictions_resampled, color=colors[2], linestyle='--')
plt.legend(loc='lower right');

Invoke plot_model_prc on the predictions of baseline model and train data and test data

In [ ]:
plot_model_prc("Train Baseline_Model", df_train_labels, train_predictions_baseline, color=colors[0])
plot_model_prc("Test Baseline_Model", df_test_labels, test_predictions_baseline, color=colors[0], linestyle='--')
plt.legend(loc='lower right');

In [ ]:
plot_model_prc("Train Weighted_Model", df_train_labels, train_predictions_weighted, color=colors[1])
plot_model_prc("Test Weighted_Model", df_test_labels, test_predictions_weighted, color=colors[1], linestyle='--')
plt.legend(loc='lower right');

In [ ]:
plot_model_prc("Train Resampled_Model", df_train_labels, train_predictions_resampled, color=colors[2])
plot_model_prc("Test Resampled_Model", df_test_labels, test_predictions_resampled, color=colors[2], linestyle='--')
plt.legend(loc='lower right');

In [ ]:
# Load the row of data you want to test
row_to_test = df.iloc[7]  # Assuming you want to test the first row

# Preprocess the row of data
# Extract features and scale them
test_features = row_to_test.drop('FLAG')  # Assuming 'FLAG' is the target variable
test_features = np.array(test_features).reshape(1, -1)  # Reshape to match the model's input shape
test_features = scale_features.transform(test_features)  # Scale features using the same scaler used in training

# Make predictions using the trained model
predictions = model.predict(test_features)

# Interpret the predictions
predicted_class = 'Fraudulent' if predictions[0] >= 0.5 else 'Non-Fraudulent'
print(f"The predicted class for the row is: {predicted_class}")